In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random
import joblib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/dataset/tfm/scraping_animeList.csv')

In [5]:
# Total de filas y columnas
df.shape

(12598, 20)

In [6]:
df.head()

,ID,Title,Type,Episodes,Source,Studios,Demographic,Year,Producers,Genres,Licensors,Duration,Rating,Score,Ranked,Popularity,Members,Favorites,Image_URL,Synopsis
0,1,Cowboy Bebop,TV,26.0,Original,Sunrise,NaN,1998,Bandai Visual,"Action, Award Winning, Sci-Fi",Funimation,24,R - 17+ (violence & profanity),8.0,46.0,43,"1,864,022","82,369",https://cdn.myanimelist.net/images/anime/4/196...,"Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,1.0,Original,Bones,NaN,2001,"Sunrise, Bandai Visual","Action, Sci-Fi","Sony Pictures Entertainment, Funimation",1,R - 17+ (violence & profanity),8.0,193.0,619,"377,963","1,581",https://cdn.myanimelist.net/images/anime/1439/...,"Another day, another bounty—such is the life o..."
2,6,Trigun,TV,26.0,Manga,Madhouse,Shounen,1998,Victor Entertainment,"Action, Adventure, Sci-Fi",Funimation,24,PG-13 - Teens 13 or older,8.0,341.0,252,"763,023","16,009",https://cdn.myanimelist.net/images/anime/7/203...,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,TV,26.0,Original,Sunrise,NaN,2002,"Bandai Visual, Dentsu, Victor Entertainment, T...","Action, Drama, Mystery, Supernatural","Funimation, Bandai Entertainment",25,PG-13 - Teens 13 or older,7.0,3073.0,1857,"116,691",648,https://cdn.myanimelist.net/images/anime/10/19...,Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,TV,52.0,Manga,Toei Animation,Shounen,2004,"TV Tokyo, Dentsu","Action, Adventure, Fantasy",Illumitoon Entertainment,23,PG - Children,6.0,4542.0,5354,"15,526",15,https://cdn.myanimelist.net/images/anime/7/215...,It is the dark century and the people are suff...


In [7]:
# Comprobamos columnas con valores faltantes
df.isna().mean()*100

ID              0.000000
Title           0.000000
Type            9.176060
Episodes        0.547706
Source          0.000000
Studios         0.000000
Demographic    66.796317
Year            0.000000
Producers       0.000000
Genres         10.501667
Licensors      68.566439
Duration        0.000000
Rating          1.270043
Score           0.000000
Ranked         18.280679
Popularity      0.000000
Members         0.000000
Favorites       0.000000
Image_URL       0.000000
Synopsis        0.023813
dtype: float64

In [8]:
# Eliminar las columnas con mayor número de valores faltantes y las IDs además eliminamos Studios y Producers ya que tienen muchos 'add some' (posible error al scrapear)
df.drop(['ID', 'Demographic', 'Licensors', 'Studios', 'Producers'], axis=1, inplace=True)

In [9]:
# Comprobamos cuantas filas con valores faltantes obtenemos
sum(df.isna().any(axis=1))

3056

In [10]:
# Procedemos a eliminarlas
df.dropna(axis=0, inplace=True)

In [11]:
df.shape

(9542, 15)

In [12]:
# Convertimos a enteros los float
df.Episodes = df.Episodes.astype(int)
df.Score = df.Score.astype(int)
df.Ranked = df.Ranked.astype(int)

In [13]:
# Reemplazamos la coma por nada y convertimos a enteros
df.Members = df.Members.str.replace(',','').astype(int)
df.Favorites = df.Favorites.str.replace(',','').astype(int)

In [14]:
# Aplicamos Label Encoder
cols = ['Type', 'Source', 'Rating']
le = LabelEncoder()
for col in cols:
  df[col] = le.fit_transform(df[col])

In [15]:
# Codificamos los géneros con 1-hot encoding
df_genres = df.Genres.str.get_dummies(sep=',')

In [16]:
# Unimos los géneros codificados al df
df_num = pd.concat([df.drop('Genres', axis=1), df_genres], axis=1)

In [17]:
# Filtramos de modo que nos quedamos con aquellas series que tienen sinopsis ya que la usaremos para medir la similitud
df_num = df_num[~df_num['Synopsis'].str.contains('No synopsis information')]

In [18]:
# Preprocesamos el texto
def preprocess_text(text):
  # Dejamos palabras
  text = ' '.join(re.findall(r'\b[a-zA-Z]+\b', text))
  # Eliminación de los espacios adicionales
  text = re.sub(r'\s+', ' ', text).strip()
  return text.lower()
df_num['Synopsis'] = df_num['Synopsis'].apply(preprocess_text)

In [19]:
# Codificamos el texto a formato numérico con tfidf
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=0.001)
X_tfidf = vectorizer.fit_transform(df_num['Synopsis'])

In [20]:
# Seleccionar las numéricas del df y escalarlas
X_num = df_num.drop(['Title', 'Image_URL', 'Synopsis'], axis=1)
X_num_sc = MinMaxScaler().fit_transform(X_num)

In [21]:
# Unir horizontalmente la matriz de tfidf y de los datos escalados
X = np.hstack((X_num_sc, X_tfidf.toarray()))
X.shape

(9062, 5373)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
# Crear un vectorizador CountVectorizer
count_vectorizer = CountVectorizer()

# Entrenar el vectorizador con los títulos de los animes
count_vectorizer.fit(df_num['Title'])

CountVectorizer()

In [23]:
# Guardar el vectorizador CountVectorizer
joblib.dump(count_vectorizer, '/content/drive/MyDrive/dataset/tfm/vectorizer.pkl')

['/content/drive/MyDrive/dataset/tfm/vectorizer.pkl']

In [24]:
# Entrenamos un modelo KNN para obtener las distancias entre series
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine").fit(X)

In [25]:
# Reseteamos el indice
df_num.reset_index(inplace=True)

In [26]:
# Función de recomendación de series
def get_movie_recommendations(title):
    movie_index = df_num[df_num["Title"] == title].index[0]
    distances, indices = model.kneighbors(np.array([X[movie_index]]))
    similar_movies = [(df_num["Title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]
get_movie_recommendations('Cowboy Bebop')

[('Code Geass: Hangyaku no Lelouch R2', 0.18510794749933235),
 ('Code Geass: Hangyaku no Lelouch', 0.18610596407991742),
 ('Tengen Toppa Gurren Lagann', 0.18752404802119038),
 ('Koukaku Kidoutai: Stand Alone Complex', 0.21540994086582532),
 ('Macross F', 0.22484044811496062)]

In [27]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9062 entries, 0 to 9061
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           9062 non-null   int64 
 1   Title           9062 non-null   object
 2   Type            9062 non-null   int64 
 3   Episodes        9062 non-null   int64 
 4   Source          9062 non-null   int64 
 5   Year            9062 non-null   int64 
 6   Duration        9062 non-null   int64 
 7   Rating          9062 non-null   int64 
 8   Score           9062 non-null   int64 
 9   Ranked          9062 non-null   int64 
 10  Popularity      9062 non-null   int64 
 11  Members         9062 non-null   int64 
 12  Favorites       9062 non-null   int64 
 13  Image_URL       9062 non-null   object
 14  Synopsis        9062 non-null   object
 15   Adventure      9062 non-null   int64 
 16   Avant Garde    9062 non-null   int64 
 17   Award Winning  9062 non-null   int64 
 18   Boys Lo

In [28]:
model

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [29]:
# Función de recomendación de series
def get_movie_recommendations(title):
    try:
        movie_index = df_num[df_num["Title"] == title].index[0]
        distances, indices = model.kneighbors(np.array([X[movie_index]]))
        similar_movies = [(df_num["Title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
        return similar_movies[1:]
    except IndexError:
        print("Este anime o bien no se encuentra en la base de datos o bien esta mal escrito.")
        print("Aquí tienes algunas recomendaciones aleatorias:")
        random_recommendations = random.sample(list(df_num["Title"]), k=5)
        return [(title, 0) for title in random_recommendations]

# Ejemplo de uso
recommendations = get_movie_recommendations('One Piece')
print(recommendations)


Este anime o bien no se encuentra en la base de datos o bien esta mal escrito.
Aquí tienes algunas recomendaciones aleatorias:
[('Osiris no Tenbin: Season 2', 0), ('Tennis no Oujisama: Mini Theater', 0), ('Bakuon!! OVA', 0), ('Shinseiki Evangelion', 0), ('Miracle Train: Chuo-sen e Youkoso', 0)]


In [30]:
# Guardar el modelo KNN
# joblib.dump(model, '/content/drive/MyDrive/dataset/tfm/modelo_knn.pkl')

# joblib.dump(vectorizer, '/content/drive/MyDrive/dataset/tfm/tfidf_vectorizer.pkl')
# joblib.dump(X_num_sc, '/content/drive/MyDrive/dataset/tfm/minmax_scaler.pkl')
# df_num.to_pickle('/content/drive/MyDrive/dataset/tfm/df_num.pkl')

# Guardar los datos preprocesados (por ejemplo, la matriz X)
# joblib.dump(X, '/content/drive/MyDrive/dataset/tfm/datos_preprocesados.pkl')

# joblib.dump(vectorizer, '/content/drive/MyDrive/dataset/tfm/vectorizer.pkl')

In [31]:
# !pip install scikit-learn==1.4.2

In [32]:
import sklearn
print(sklearn.__version__)


1.2.2


In [33]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.neighbors import NearestNeighbors
import random
import joblib

df = pd.read_csv('/content/drive/MyDrive/dataset/tfm/scraping_animeList.csv')

# Comprobamos columnas con valores faltantes
df.isna().mean()*100

# Eliminar las columnas con mayor número de valores faltantes y las IDs además eliminamos Studios y Producers ya que tienen muchos 'add some' (posible error al scrapear)
df.drop(['ID', 'Demographic', 'Licensors', 'Studios', 'Producers'], axis=1, inplace=True)

# Comprobamos cuantas filas con valores faltantes obtenemos
sum(df.isna().any(axis=1))

# Procedemos a eliminarlas
df.dropna(axis=0, inplace=True)

# Convertimos a enteros los float
df.Episodes = df.Episodes.astype(int)
df.Score = df.Score.astype(int)
df.Ranked = df.Ranked.astype(int)

# Reemplazamos la coma por nada y convertimos a enteros
df.Members = df.Members.str.replace(',','').astype(int)
df.Favorites = df.Favorites.str.replace(',','').astype(int)

# Aplicamos Label Encoder
cols = ['Type', 'Source', 'Rating']
le = LabelEncoder()
for col in cols:
  df[col] = le.fit_transform(df[col])

# Codificamos los géneros con 1-hot encoding
df_genres = df.Genres.str.get_dummies(sep=',')

# Unimos los géneros codificados al df
df_num = pd.concat([df.drop('Genres', axis=1), df_genres], axis=1)

# Filtramos de modo que nos quedamos con aquellas series que tienen sinopsis ya que la usaremos para medir la similitud
df_num = df_num[~df_num['Synopsis'].str.contains('No synopsis information')]

# Preprocesamos el texto
def preprocess_text(text):
    # Dejamos solo palabras alfabéticas
    text = ' '.join(re.findall(r'\b[a-zA-Z]+\b', str(text)))
    # Eliminamos espacios adicionales
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

df_num['Synopsis'] = df_num['Synopsis'].apply(preprocess_text)

# Codificamos el texto a formato numérico con tfidf
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=0.001)
X_tfidf = vectorizer.fit_transform(df_num['Synopsis'])

# Seleccionar las numéricas del df y escalarlas
X_num = df_num.drop(['Title', 'Image_URL', 'Synopsis'], axis=1)
X_num_sc = MinMaxScaler().fit_transform(X_num)

# Unir horizontalmente la matriz de tfidf y de los datos escalados
X = np.hstack((X_num_sc, X_tfidf.toarray()))
X.shape

# Entrenamos un modelo KNN para obtener las distancias entre series
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine").fit(X)

# Reseteamos el índice
df_num.reset_index(inplace=True)

# Guardar el modelo KNN
# joblib.dump(model, '/content/drive/MyDrive/dataset/tfm/modelo_knn2.pkl')
# joblib.dump(vectorizer, '/content/drive/MyDrive/dataset/tfm/vectorizer2.pkl')